## Skoltech HackRace Hackaton Final Round (First place solution)
#### Emil Kayumov (MSU), Alexander Nenashev (MIPT)


Reading data.

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
orders = pd.read_csv("order.csv", header=None)
times = np.load('times.npy')

In [3]:
orders = np.array(orders)

In [4]:
orders

array([[    0,  2643,  2642,     0],
       [    1,  2643,     2,     5],
       [    2,   535,  1126,    10],
       ..., 
       [ 2997,   561,  2021, 14985],
       [ 2998,  2544,  1784, 14990],
       [ 2999,  1217,   149, 14995]])

In [5]:
times

array([[    0.  ,   369.75,   472.9 , ...,  2600.08,  2523.55,  1737.76],
       [  320.62,     0.  ,   103.15, ...,  2436.68,  2420.03,  1634.25],
       [  385.66,    83.15,     0.  , ...,  2500.64,  2483.99,  1698.2 ],
       ..., 
       [ 2584.5 ,  2567.89,  2611.85, ...,     0.  ,  4673.33,  3887.54],
       [ 2443.68,  2503.85,  2547.81, ...,  4530.99,     0.  ,  3427.75],
       [ 1602.93,  1633.27,  1737.44, ...,  3721.96,  3324.91,     0.  ]])

For example lets combine order 0 and order 1.

In [6]:
times[2643, 2642] + times[2643, 2] # separately

5062.3500000000004

In [7]:
times[2643, 2] + times[2, 2642] # 0-1-1-0

4221.4300000000003

Simple solution.

In [8]:
# check if can cobmine order i and order j

def find_route(i, j):
    
    start_i = orders[i, 1]
    end_i = orders[i, 2]
    
    start_j = orders[j, 1]
    end_j = orders[j, 2]
    
    start_i_start_j = times[start_i, start_j]
    start_i_end_j = times[start_i, end_j]
    
    start_j_start_i = times[start_j, start_i]
    start_j_end_i = times[start_j, end_i]
    
    start_i_end_i = times[start_i, end_i]
    start_j_end_j = times[start_j, end_j]
    
    end_i_end_j = times[end_i, end_j]
    end_j_end_i = times[end_j, end_i]
    
    
    min_time = times[orders[i, 1], orders[i, 2]] + times[orders[j, 1], orders[j, 2]]
    acts = (i, i, j, j)
    start_time = orders[i, 3]
    
    # i -> j
    diff = start_i_start_j - (orders[j, 3] - orders[i, 3])
    if diff < 900 and diff > -900:
        # can take i -> j
        
        # ... -> i -> j
        tmp_time = start_i_start_j + start_j_end_i + end_i_end_j
        if tmp_time < min_time:
            min_time = tmp_time
            acts = (i, j, i, j)
            if diff < 0:
                start_time = orders[i, 3] + np.abs(diff)
    
        # ... -> j -> i
        tmp_time = start_i_start_j + start_j_end_j + end_j_end_i
        if tmp_time < min_time:
            min_time = tmp_time
            acts = (i, j, j, i)
            if diff < 0:
                start_time = orders[i, 3] + np.abs(diff)
                
    # j -> i
    diff = start_i_start_j + (orders[j, 3] - orders[i, 3]) 
    if diff < 900 and diff > -900:
        # can take j -> i
        
        # ... -> j -> i
        tmp_time = start_i_start_j + start_i_end_j + end_j_end_i
        if tmp_time < min_time:
            min_time = tmp_time
            acts = (j, i, j, i)
            start_time = orders[j, 3] 
            
        # ... -> i -> j
        tmp_time = start_i_start_j + start_i_end_i + end_i_end_j
        if tmp_time < min_time:
            min_time = tmp_time
            acts = (j, i, i, j)
            start_time = orders[j, 3] 
            
    return min_time, acts, start_time

In [9]:
print(find_route(0, 1))

(4221.4300000000003, (0, 1, 1, 0), 5.0)


In [10]:
output = open('answer.txt', 'w')
orders_done = np.zeros(orders.shape[0], dtype=bool)

for i in range(3000):
    
    if orders_done[i]:
        continue
    
    min_time = 1e100
    best_pair = (i, i)
    best_acts = (i, i, 0, 0)
    start_time = -1
    
    for j in range(i + 1, orders.shape[0]):
        if orders_done[j]:
            continue
        cur_time, cur_acts, cur_start = find_route(i, j)
        if cur_time < min_time and cur_acts[0] != cur_acts[1]:
            min_time = cur_time
            best_pair = (i, j)
            best_acts = cur_acts
            start_time = cur_start
    
    # write only combined orders
    if best_acts[0] != best_acts[1]:
        
        line = str(math.ceil(start_time)) + ' ' 

        if orders[best_acts[0], 1] == orders[best_acts[1], 1]:
            # equal start point
            line += str(orders[best_acts[0], 1]) + ' ' + str(best_acts[0]) + ' ' + str(best_acts[1]) + ','
        else:
            line += str(orders[best_acts[0], 1]) + ' ' + str(best_acts[0]) + ',' + str(orders[best_acts[1], 1]) + ' ' + str(best_acts[1]) + ','

        if orders[best_acts[0], 2] == orders[best_acts[1], 2]:
            # equal finish point
            line += str(orders[best_acts[2], 2]) + ' ' + str(best_acts[2]) + ' ' + str(best_acts[3]) + '\n'
        else:
            line += str(orders[best_acts[2], 2]) + ' ' + str(best_acts[2]) + ',' + str(orders[best_acts[3], 2]) + ' ' + str(best_acts[3]) + '\n'

        output.writelines(line)

        orders_done[i] = True
        orders_done[j] = True
        
    else:
        orders_done[i] = True
        
output.close()

Result:

In [12]:
!python check.py

576476.8
